In [7]:
def compose_instruction(task,
                        output_dir,
                        num_train_epochs = '3',
                        learning_rate = '1e-5',
                        training_size = '1',
                        insert_dropout = '-1',
                        catch_dropout = '-1',
                        use_modded = False,
                        model_name_or_path="bert-base-cased",
                        max_length = '128',
                        per_device_train_batch_size = 32,
                        seed = '0',
                        lr_scheduler_type = "linear",
                        num_warmup_steps = 200,
                        early_stopping_patience=100,
                        early_stopping_min_delta=0
                        ):
    

    base_command = 'accelerate launch run_glue_no_trainer_modded.py '
    per_device_train_batch_size = 16
    # base_command = 'nohup python run_glue_no_trainer_modded.py '
    argument_list = ["--model_name_or_path", model_name_or_path,
                     "--max_length", max_length,
                     "--per_device_train_batch_size", f'{per_device_train_batch_size}',
                     "--seed", seed,
                     "--with_tracking",
                     "--report_to", "all"
                    ]

    argument_list = argument_list + ["--lr_scheduler_type" , f"{lr_scheduler_type}"]
    argument_list = argument_list + ["--num_warmup_steps" , f"{num_warmup_steps}"]
    argument_list = argument_list + ["--early_stopping_patience", f"{early_stopping_patience}"]
    argument_list = argument_list + ["--early_stopping_min_delta", f"{early_stopping_min_delta}"]


    argument_list = argument_list + ["--task_name", f'{task}']
    argument_list = argument_list + ["--num_train_epochs", f'{num_train_epochs}']
    argument_list = argument_list + ["--learning_rate", f'{learning_rate}']
    argument_list = argument_list + ["--training_size", f'{training_size}']
    argument_list = argument_list + ["--insert_dropout", f'{insert_dropout}']
    argument_list = argument_list + ["--catch_dropout", f'{catch_dropout}']


    if use_modded:
        argument_list.append("--use_modded")


    argument_list = argument_list + ["--output_dir",output_dir]
    argument_string = ' '.join(argument_list)
    instruction = base_command + argument_string
    return instruction

# Reference Experiments

In [8]:
experiment_parameters_reference = {
    'model_name_or_path':["bert-base-cased"],
    "task" : ['rte'],
    "learning_rate" : [1e-6,2e-7,5e-7],
    "insert_dropout" : [0.1],
    "training_size" : [32],
    "num_warmup_steps" : [500],
    "use_modded" : [False],
    "num_train_epochs" : [4096]
}

In [9]:
from itertools import product
keys, values = zip(*experiment_parameters_reference.items())
permutations_dicts_reference = [dict(zip(keys, v)) for v in product(*values)]
print(len(permutations_dicts_reference))
permutations_dicts_reference[0]

3


{'model_name_or_path': 'bert-base-cased',
 'task': 'rte',
 'learning_rate': 1e-06,
 'insert_dropout': 0.1,
 'training_size': 32,
 'num_warmup_steps': 500,
 'use_modded': False,
 'num_train_epochs': 4096}

# Fantastic-Umbrella Experiments

In [10]:
experiment_parameters_modded = {
    'model_name_or_path':["bert-base-cased"],
    "task" : ['rte'],
    "learning_rate" : [1e-6, 5e-7, 2e-7],
    "insert_dropout" : [0.2],
    "catch_dropout" : [0.0],
    "training_size" : [32],
    "use_modded" : [True],
    "num_train_epochs" : [4096],
    "num_warmup_steps" : [500]
}

In [11]:
from itertools import product
keys, values = zip(*experiment_parameters_modded.items())
permutations_dicts_modded = [dict(zip(keys, v)) for v in product(*values)]
print(len(permutations_dicts_modded))
permutations_dicts_modded[0]

1


{'model_name_or_path': 'bert-base-cased',
 'task': 'rte',
 'learning_rate': 1e-06,
 'insert_dropout': 0.2,
 'catch_dropout': 0.0,
 'training_size': 32,
 'use_modded': True,
 'num_train_epochs': 4096,
 'num_warmup_steps': 500}

# Generate instructions

In [12]:
generation = "11a"
base_dir = '/home/lange/fantastic-umbrella/runs'

from math import log, ceil
# permutations_dicts_modded = []
permutations_dicts_reference = []
permutations_dicts = permutations_dicts_reference + permutations_dicts_modded
magnitude = ceil(log(len(permutations_dicts),10))


with open(f'instructions_{generation}.txt', 'w', newline='\n') as f:
    
    for idx, parameters in enumerate(permutations_dicts):
        # print(f'##################################### Current iteration: {idx} #####################################')
        
        run_id = str(idx).zfill(magnitude)
        if parameters["use_modded"]:
            run_type = "modded"
        else:
            run_type = "vanilla"

        run_name = f'run_{generation}_{run_id}_{run_type}'

        output_dir = f'{base_dir}/{run_name}'
        instruction = compose_instruction(output_dir=output_dir,
                                          **parameters)
        f.write(instruction + '\n')
